# Infer SOMA -  2️⃣
 🚧 WIP 🚧 (🚨🚨🚨🚨 Steps 3-9 depend on establishing a good solution here.)

--------------

## OBJECTIVE: ✅ Infer sub-cellular component SOMA in order to understand interactome 

>> WARNING:  THIS IS NOT WORKING

>> Currently it just returns the entire image as SOMA.

To measure shape, position, and size of the cell body -- the soma.    There are a variety of signals from which we could make this inference.  The two most promising are a composite signal including the residual from linear unmixing (e.g. `ch = [1, 4, 5,7]`) and a signal derived from the lysosome channel (`ch = 1`).    In all procdures scaling the intensities to find the lower florescence signals at the edge of the soma against baseline is employed.  

In the long term we can build of a database of "ground truth" by sourcing additional markers which can be iteratively improved.  For example using the Allen Cell "Label Free" segmentation results should provide a good corroboration or constraints to the procedures outlined below.  

Three possible _workflows_ are illustrated below.

Dependencies:
The CYTOSOL inference rely on the NUCLEI AND SOMA inference.  Therefore all of the sub-cellular objects rely on this segmentation.





# IMPORTS

In [47]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict

import numpy as np
import scipy

# TODO:  prune the imports.. this is the big set for almost all organelles
# # function for core algorithm
from scipy import ndimage as ndi
import aicssegmentation
from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.utils import topology_preserving_thinning, size_filter
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.utils import hole_filling
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice
from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation, remove_small_holes   # function for post-processing (size filter)
from skimage.measure import label

# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc_2d
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc_2d.utils.file_io import (read_input_image, 
                                                                    list_image_files, 
                                                                    export_ome_tiff, 
                                                                    etree_to_dict, 
                                                                    save_parameters, 
                                                                    load_parameters, 
                                                                    export_ndarray)
from infer_subc_2d.utils.img import *
from infer_subc_2d.organelles.nuclei import infer_NUCLEI

%load_ext autoreload
%autoreload 2

test_img_n = 5

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# IMAGE PROCESSING  OBJECTIVE :  infer SOMA
 
> #### Note:  we are using the Nuclei of the brightest cell to aid in inferring the Soma and Cytosol objects.   Because we do NOT have a direct cell membrane / soma signal this is the trickiest and potentially problematic part of the overall sub-cellular component inference.   The Soma (via the Cytosol mask) will be used to define ALL subsequent sub-cellular Objects.

------------------------
# LOAD RAW IMAGE DATA
Identify path to _raw_ image data and load our example image


In [26]:
# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files
img_file_list = list_image_files(data_path,im_type)
test_img_name = img_file_list[test_img_n]


In [27]:

bioim_image = read_input_image(test_img_name)
img_data = bioim_image.image
raw_meta_data = bioim_image.raw_meta
ome_types = []
meta_dict = bioim_image.meta

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


## Get default parameters, including  "optimal" Z

takes ~ 4 seconds to calculate

In [45]:
load_Z_from_params = False


if load_Z_from_params:

    default_params = load_parameters( test_img_name.split("/")[-1], data_root_path / "intermediate" )

    ch_to_agg = default_params["ch_to_agg"]
    nuc_ch = default_params['nuc_ch']
    optimal_Z = default_params["optimal_Z"] #find_optimal_Z(img_data, nuc_ch, ch_to_agg) 
else:
    ch_to_agg = (1,2,3,4,5,6)
    nuc_ch = 0
    optimal_Z = find_optimal_Z(img_data, nuc_ch, ch_to_agg) 

    default_params = defaultdict(str, **{
        #"intensity_norm_param" : [0.5, 15]
        "intensity_norm_param" : [0],
        "gaussian_smoothing_sigma" : 1.34,
        "gaussian_smoothing_truncate_range" : 3.0,
        "dot_2d_sigma" : 2,
        "dot_2d_sigma_extra" : 1,
        "dot_2d_cutoff" : 0.025,
        "min_area" : 10,
        "low_level_min_size" :  100,
        "median_filter_size" : 4,
        "ch_to_agg" : (1,2,3,4,5,6), # exclude residual
        "nuc_ch" : 0,
        "optimal_Z": optimal_Z,
    })
    save_parameters(default_params, test_img_name.split("/")[-1], data_root_path / "intermediate" )
# make sure we have removed Z
if len(scale)>2:
    scale = scale[1:]


## get the inferred nuclei object

(takes < 1 sec)

In [48]:


img_2D = img_data[:,[optimal_Z],:,:].copy()

raw_nuclei = img_2D[0].copy()
NU_object, NU_label, out_p =  infer_NUCLEI(raw_nuclei.copy(), default_params) 


intensity normalization: min-max normalization with NO absoluteintensity upper bound


# IMAGE PROCESSING PROTOTYPE, Objective 2:  infer SOMA

>WORKFLOW #1  - modified MCZ 3/20

Segmentation on a 3 channel composite as per 3/20 pipeline from MCZ
Summary - Starting with a linear combination of three signals,  the signal is smoothed and non-linearly combined (logrithmic and edge detected) for thresholding. 
## summary of steps

➡️ INPUT
- multi-channel sum (4*1,5,7)
- labeled NUCLEI (objective #1)

PRE-PROCESSING
- ne-noise and somoothe
- log transform inensities
- scale to max 1.0
- create non-linear aggregate of log-intensity + scharr filtered 

CORE PROCESSING
- mask object segmentation at bottom

POST-PROCESSING
  - fill holes
  - remove small objects

POST-PROCESSING
  - keep only the "most intense" Soma


OUTPUT ➡️ 
- mask of SOMA


In [49]:
scale, NU_label.shape

((0.07987165184837318, 0.07987165184837318), (1, 768, 768))

### INPUT

In [50]:

###################
# INPUT
###################
struct_img_raw = (4. * img_2D[1].copy() + 
                               1. * img_2D[5].copy() + 
                               1. * img_2D[7].copy() )



### PRE-PROCESSING


In [51]:

###################
# PRE_PROCESSING
###################
#
# DEFAULT PARAMETERS:
#intensity_norm_param = [0.5, 15]
scaling_param = [0]
gaussian_smoothing_sigma = 1.
gaussian_smoothing_truncate_range = 3.0

med_filter_size = 15  

# Linear-ish smoothing
# raw_soma_linear = intensity_normalization( struct_img_raw ,  scaling_param=scaling_param)
raw_soma_linear = simple_intensity_normalization(struct_img_raw.copy())


# # this is SLOOOOOW.... 19 seconds!!!!
# struct_img1 = ndi.median_filter(raw_soma_linear, size=med_filter_size)

# hack it to speed up 10X
struct_img = ndi.median_filter(raw_soma_linear.squeeze(), size=med_filter_size)[np.newaxis,:,:]



structure_img_smooth = ndi.gaussian_filter( struct_img,
                                                                                sigma=gaussian_smoothing_sigma,
                                                                                mode="nearest", 
                                                                                truncate=gaussian_smoothing_truncate_range,
                                                                                )


# NON-Linear aggregation
log_image, d = log_transform( structure_img_smooth ) 
log_image = intensity_normalization(  log_image,  scaling_param=[0] )

edges = filters.scharr(log_image)

composite_soma = intensity_normalization(  edges,  scaling_param=[0] ) + log_image 


intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound


### CORE PROCESSING

In [52]:
###################
# CORE_PROCESSING
###################
low_level_min_size =  100


# "Masked Object Thresholding" - 3D
bw, _bw_low_level = MO(composite_soma, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= 0.5, 
                                                return_object=True,
                                                dilate=True)
                                                


### POST-PROCESSING

In [53]:
###################
# POST_PROCESSING
###################

# 2D 
hole_width = 80
removed_holes = remove_small_holes(bw, hole_width ** 2 )

small_object_max = 45
cleaned_img = size_filter_2D(removed_holes, 
                                                            min_size= small_object_max**2, 
                                                            connectivity=1)

# limit the labeling to where we have soma or NUclear signal
# watershed_mask = np.logical_or(cleaned_img, NU_labels > 0)
watershed_mask = cleaned_img 
inverted_img = 1. - composite_soma

labels_out = watershed(
            connectivity=np.ones((1, 3,3), bool),
            image=inverted_img,
            markers=NU_label,
            mask=watershed_mask,
            )



In [54]:
labels_out.shape, NU_label.shape

((1, 768, 768), (1, 768, 768))

In [55]:
###################
# POST- POST_PROCESSING
###################
# keep the "SOMA" label which contains the highest total signal
all_labels = np.unique(labels_out)[1:]

total_signal = [ raw_soma_linear[labels_out == label].sum() for label in all_labels]
# combine NU and "labels" to make a SOMA
keep_label = all_labels[np.argmax(total_signal)]
keep_label

soma_out = np.zeros_like(labels_out)
soma_out[labels_out==keep_label] = 1

#### Visualize with `napari`
Visualize the first-pass segmentation and labeling with `napari`.

In [56]:

viewer = napari.view_image(
        cleaned_img,
        scale=scale
    )


viewer.scale_bar.visible = True
viewer.add_labels(
    labels_out,
    scale=scale 
)

viewer.add_labels(
    soma_out,
    scale=scale 
)

<Labels layer 'soma_out' at 0x18954a940>

## DEFINE `_infer_SOMA` function

Based on the _prototyping_ above define the function to infer soma.  

> NOTE:  although it takes the parameters as input, they are all "hard coded" below, and the function returns the parameters in the same `defaultdict`

In [57]:
##########################
# 2. infer_SOMA
##########################
def _infer_SOMA(struct_img: np.ndarray, NU_labels: np.ndarray,  in_params:dict) -> tuple:
    """
    Procedure to infer SOMA from linearly unmixed input.

    Parameters:
    ------------
    struct_img: np.ndarray
        a 3d image containing the SOMA signal

    NU_labels: np.ndarray boolean
        a 3d image containing the NU labels

    in_params: dict
        holds the needed parameters

    Returns:
    -------------
    tuple of:
        object
            mask defined boundaries of SOMA
        label
            label (could be more than 1)
        parameters: dict
            updated parameters in case any needed were missing
    
    """
    out_p= in_params.copy()

    ###################
    # PRE_PROCESSING
    ###################                         

    #TODO: replace params below with the input params
    scaling_param =  [0]   
    struct_img = intensity_normalization(struct_img, scaling_param=scaling_param)
    out_p["intensity_norm_param"] = scaling_param

    # make a copy for post-post processing
    scaled_signal = struct_img.copy()

    # Linear-ish processing
    med_filter_size = 15   
    # structure_img_median_3D = ndi.median_filter(struct_img,    size=med_filter_size  )
    struct_img = ndi.median_filter(struct_img.squeeze(), size=med_filter_size)[np.newaxis,:,:]

    out_p["median_filter_size"] = med_filter_size 
    gaussian_smoothing_sigma = 1.
    gaussian_smoothing_truncate_range = 3.0

    struct_img = ndi.gaussian_filter( struct_img,
                                                                                sigma=gaussian_smoothing_sigma,
                                                                                mode="nearest", 
                                                                                truncate=gaussian_smoothing_truncate_range,
                                                                                )


    out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
    out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range

    # non-Linear processing
    log_img, d = log_transform( struct_img ) 
    log_img = intensity_normalization(  log_img,  scaling_param=[0] )

    struct_img = intensity_normalization(  filters.scharr(log_img),  scaling_param=[0] )  + log_img


    ###################
    # CORE_PROCESSING
    ###################
    local_adjust = 0.5
    low_level_min_size = 100
    # "Masked Object Thresholding" - 3D
    struct_obj, _bw_low_level = MO(struct_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= local_adjust, 
                                                return_object=True,
                                                dilate=True)

    out_p["local_adjust"] = local_adjust 
    out_p["low_level_min_size"] = low_level_min_size 

    ###################
    # POST_PROCESSING
    ###################

    # 2D 
    hole_max = 80  
    # struct_obj = hole_filling(struct_obj, hole_min =0. , hole_max=hole_max**2, fill_2d = True) 
    struct_obj = remove_small_holes(struct_obj, hole_max ** 2 )

    out_p['hole_max'] = hole_max

    small_object_max = 35
    struct_obj = size_filter_2D(struct_obj, 
                                                            min_size= small_object_max**2, 
                                                            connectivity=1)
    out_p['small_object_max'] = small_object_max

    labels_out = watershed(
                connectivity=np.ones((1, 3,3), bool),
                image=1. - struct_img,
                markers=NU_labels,
                mask= np.logical_or(struct_obj, NU_labels > 0), #struct_obj
                )



                # 2D 

    ###################
    # POST- POST_PROCESSING
    ###################
    # keep the "SOMA" label which contains the highest total signal
    all_labels = np.unique(labels_out)[1:]

    total_signal = [ scaled_signal[labels_out == label].sum() for label in all_labels]
    # combine NU and "labels" to make a SOMA
    keep_label = all_labels[np.argmax(total_signal)]

    soma_out = np.zeros_like(struct_obj)
    soma_out[labels_out==keep_label] = 1

    retval = (struct_obj,  soma_out , out_p)
    return retval




---------------------
# TEST `_infer_SOMA`  function defined above


##


In [59]:

###################
# SOMA INPUT
###################

img_2D = img_data[:,[optimal_Z],:,:].copy()
raw_soma = (4. * img_2D[1].copy() + 
                               1. * img_2D[5].copy() + 
                               1. * img_2D[7].copy() )





In [60]:
NU_label.shape, optimal_Z

((1, 768, 768), 10)

In [61]:

SO_object, SO_label, out_p =  _infer_SOMA(raw_soma.copy(), NU_label, default_params) 

intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound


In [42]:
viewer = napari.view_image(
    raw_nuclei,
    scale=scale,
    blending='additive'
)
viewer.scale_bar.visible = True

viewer.add_image(
    NU_object,
    scale=scale,
    colormap='blue', 
    blending='additive'
)

viewer.dims.ndisplay = 2
viewer.camera.angles = (-30, 25, 120)

viewer.add_image(
    raw_soma,
    scale=scale,
    blending='additive'
)

viewer.add_image(
    SO_object,
    scale=scale,
    blending='additive'
)


<Image layer 'SO_object' at 0x17c179f10>

--------------------------

# TEST `infer_SOMA` exported functions


##
`infer_SOMA` procedure

In [ ]:

SO_object, SO_label, out_p =  _infer_SOMA(raw_soma.copy(), NU_label, default_params) 

In [ ]:
from infer_subc_2d.organelles.soma import infer_SOMA

SO_object, SO_label, out_p =  infer_SOMA(struct_img.copy(), NU_label, default_params) 
# TODO:  make export ome_tiff export:   XX_object, XX_label, XX_signal
#              also fix Path vs. str action for export wrapper


In [62]:
# save updated parameters for ongoing testing
save_parameters(default_params, test_img_name.split("/")[-1], data_root_path / "intermediate" )

'/Users/ahenrie/Projects/Imaging/data/intermediate/ZSTACK_PBTOhNGN2hiPSCs_BR3_N14_Unmixed.czi_params.pkl'